Мета-информация в отдельной таблице:

    1 колонка - путь к файлу
    2 колонка - ресурс
    3 колонка - дата
    4 колонка - заголовок
    5 колонка - подзаголовок
    6 колонка - автор
    7 колонка - рубрика
    8 колонка - тэги(?)


In [2]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [3]:
column_names = ['link', 'source', 'date', 'title', 'subtitle', 'author', 'rubrics', 'tags']

# Nplus1 Parsing BeautifulSoup #

In [4]:
import re
from bs4 import BeautifulSoup

In [5]:
def parse_metadata_nplus1(file_path):
    ready_df = pd.DataFrame()
        
    with open(file_path) as fl:
        source = fl.read()
        
 #   path_index_yd = 'https://disk.yandex.ru/client/disk/popular_science_texts_store/'
        
    re_link = re.compile('data\/(.*)\.html')
    link = re_link.findall(line)
    link = ''.join(link) 
    
 #   path_yd = path_index_yd + link + '.txt'
 #   print(path_yd)
    
    web_source = 'nplus1.ru'
    
    soup_article = BeautifulSoup(source, 'html.parser')
    
    try:
    
        head_title = soup_article.find('h1').text.strip()
        
    except AttributeError:
        head_title = ''
        
    
    try:
        re_author = re.compile('[A-Za-z]')
        author = soup_article.find(attrs={'name': 'mediator_author'})['content']
        if re_author.match(author):
            author = ''
    except TypeError:
        author = ''
    
    subtitle = ''

    rubrics = ''
    
    try:
        broad_rubrics = soup_article.find('p', class_='table').text 
        broad_rubrics = broad_rubrics.split('\n')
        broad_rubrics = '_'.join(broad_rubrics)
    except AttributeError:
        broad_rubrics = ''

    raw_date = soup_article.find(attrs={'itemprop': 'datePublished'})['content'] 

    df = pd.DataFrame([[link, web_source, raw_date, head_title, subtitle, author,
                        broad_rubrics, rubrics]], columns=column_names)
    
   # save_clear_text(clear_p, file_path.replace('pages_politru_lectures/',''))
    
    ready_df = ready_df.append(df)
    return ready_df

# Nplus1 news #

In [58]:
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/nplus1_news_index.txt') as ps_index:
    read_index = ps_index.readlines() 

directory_index = '/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/'
path_index_yd = 'https://disk.yandex.ru/client/disk/popular_science_texts_store/'

In [64]:
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/crawlers/nplus1/nplus1_news_index_yd.txt') as n1_index_yd:
    read_index_yd = n1_index_yd.readlines()
yd_path_list = []
for line in read_index_yd:
    path_yd = path_index_yd +line[2:].rstrip()
    yd_path_list.append(path_yd)

In [67]:
data_frame = pd.DataFrame()
df_nplus1_news = pd.DataFrame(data_frame, columns=column_names)
for line in tqdm(read_index[0:10]):
    directory = directory_index + line.rstrip()
    data_frame = parse_metadata_nplus1(directory)
    df_nplus1_news = df_nplus1_news.append(data_frame)

A Jupyter Widget

In [68]:
df_nplus1_news

,link,source,date,title,subtitle,author,rubrics,tags
0,nplus1.ru/news/2015/03/02/bionics,nplus1.ru,2015-03-02,Травму нервного плечевого сплетения научились ...,,,_Наука_Разное_,
0,nplus1.ru/news/2015/03/02/blackhole,nplus1.ru,2015-03-02,Рекордно молодая сверхмассивная дыра удивила ф...,,,_Космос_Наука_Разное_,
0,nplus1.ru/news/2015/03/02/chaotic,nplus1.ru,2015-03-02,Физики поделили экономики на «ламинарные» и «х...,,,_Наука_Физики смогли_,
0,nplus1.ru/news/2015/03/02/culture,nplus1.ru,2015-03-02,Культурные различия предложили изучать анализо...,,,_Наука_Гаджеты_Разное_,
0,nplus1.ru/news/2015/03/02/ebola,nplus1.ru,2015-03-02,Известное в традиционной медицине вещество ока...,,,_Наука_Разное_,
0,nplus1.ru/news/2015/03/02/epigenom,nplus1.ru,2015-03-02,Составлена рекордно подробная карта эпигенома ...,,,_Наука_Биология_,
0,nplus1.ru/news/2015/03/02/full,nplus1.ru,2015-03-02,Химики рассказали о свойствах «волейбольного а...,,,_Наука_Технологии_Органическое на 100% _,
0,nplus1.ru/news/2015/03/02/gene,nplus1.ru,2015-03-02,Человеческий ген создал извилины в мозге мышей,,,_Наука_Биология_,
0,nplus1.ru/news/2015/03/02/goats,nplus1.ru,2015-03-02,Физики разобрались с движением баранов сквозь ...,,,_Наука_Физики смогли_,
0,nplus1.ru/news/2015/03/02/hair,nplus1.ru,2015-03-02,Преступников идентифицируют по краске для волос,,,_Наука_Разное_,


In [217]:
unique_rubrics = []
rubrics_list = df_nplus1_news['rubrics'].values
rubrics_list = rubrics_list.tolist()
for line in tqdm(rubrics_list):
    split_rubrics = line.split('_')
    split_rubrics = split_rubrics[1:-1]
    unique_rubrics.extend(split_rubrics)
unique_rubrics = set(unique_rubrics)
unique_rubrics = '\n'.join(unique_rubrics)

A Jupyter Widget

In [218]:
with open('nplus1_news_rubric_list.txt', 'w') as fl:
    fl.write(unique_rubrics)

In [317]:
df_nplus1_news_csv = df_nplus1_news.to_csv()

In [318]:
with open('nplus1_news.csv', 'w') as fl:
    fl.write(df_nplus1_news_csv)
    

# Nplus1 materials #

In [319]:
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/nplus1_material_index.txt') as ps_index:
    read_index = ps_index.readlines() 

directory_index = '/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/'

In [320]:
data_frame = pd.DataFrame()
df_nplus1_materials = pd.DataFrame(data_frame, columns=column_names)
for line in tqdm(read_index):
    directory = directory_index + line.rstrip()
    data_frame = parse_metadata_nplus1(directory)
    df_nplus1_materials = df_nplus1_materials.append(data_frame)

A Jupyter Widget

In [321]:
df_nplus1_materials

,link,source,date,title,subtitle,author,rubrics,tags
0,nplus1.ru/material/2015/04/10/waranimals,nplus1.ru,2015-04-10,"Вы звери, господа",,Александр Ершов,_Наука_Археология_,
0,nplus1.ru/material/2015/04/11/greencloud,nplus1.ru,2015-04-11,Зеленые рукава галактик,,Владимир Королев,_Космос_,
0,nplus1.ru/material/2015/04/11/seasontwo,nplus1.ru,2015-04-11,Коллайдер: Перезагрузка,,Александр Ершов,_Наука_Второй сезон Коллайдера_,
0,nplus1.ru/material/2015/04/15/just-read-the-in...,nplus1.ru,2015-04-15,Посадка Falcon 9 на платформу,,Николай Воронцов,_Космос_Мезонин_,
0,nplus1.ru/material/2015/04/15/nasa-vehicle,nplus1.ru,2015-04-15,Автомобиль нового поколения по версии NASA,,Николай Воронцов,_Блоги_Мезонин_,
0,nplus1.ru/material/2015/04/15/uniquegallery,nplus1.ru,2015-04-15,Облачный атлас,,Аня Савченко,_Наука_И о погоде_,
0,nplus1.ru/material/2015/04/16/actinides,nplus1.ru,2015-04-16,Элемент неожиданности,,Владимир Королев,_Наука_Разное_,
0,nplus1.ru/material/2015/04/16/moon-maps,nplus1.ru,2015-04-16,Лунные карты,,Николай Воронцов,_Блоги_Мезонин_,
0,nplus1.ru/material/2015/04/17/gallery,nplus1.ru,2015-04-17,«Нужно больше минералов!»,,Аня Савченко,_Наука_Разное_,
0,nplus1.ru/material/2015/04/20/celldivision,nplus1.ru,2015-04-20,Делим на два,,Аня Савченко,_Наука_,


In [251]:
unique_rubrics = []
rubrics_list = df_nplus1_materials['rubrics'].values
rubrics_list = rubrics_list.tolist()
for line in tqdm(rubrics_list):
    split_rubrics = line.split('_')
    split_rubrics = split_rubrics[1:-1]
    unique_rubrics.extend(split_rubrics)
unique_rubrics = set(unique_rubrics)
unique_rubrics = '\n'.join(unique_rubrics)

A Jupyter Widget

In [253]:
with open('nplus1_materials_rubric_list.txt', 'w') as fl:
    fl.write(unique_rubrics)

In [323]:
df_nplus1_materials_csv = df_nplus1_materials.to_csv()
with open('nplus1_materials.csv', 'w') as fl:
    fl.write(df_nplus1_materials_csv)

# Nplus1 Blog #

In [324]:
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/nplus1_blog_index.txt') as ps_index:
    read_index = ps_index.readlines() 

directory_index = '/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/'

In [327]:
data_frame = pd.DataFrame()
df_nplus1_blogs = pd.DataFrame(data_frame, columns=column_names)
for line in tqdm(read_index):
    directory = directory_index + line.rstrip()
    data_frame = parse_metadata_nplus1(directory)
    df_nplus1_blogs = df_nplus1_blogs.append(data_frame)

A Jupyter Widget

In [329]:
unique_rubrics = []
rubrics_list = df_nplus1_blogs['rubrics'].values
rubrics_list = rubrics_list.tolist()
for line in tqdm(rubrics_list):
    split_rubrics = line.split('_')
    split_rubrics = split_rubrics[1:-1]
    unique_rubrics.extend(split_rubrics)
unique_rubrics = set(unique_rubrics)
unique_rubrics = '\n'.join(unique_rubrics)

A Jupyter Widget

In [330]:
with open('nplus1_blogs_rubric_list.txt', 'w') as fl:
    fl.write(unique_rubrics)

In [331]:
df_nplus1_blogs_csv = df_nplus1_blogs.to_csv()
with open('nplus1_blogs.csv', 'w') as fl:
    fl.write(df_nplus1_blogs_csv)

# ProScience Parsing #

In [7]:
import re
import bs4 as bs
import pandas as pd

In [8]:
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/polit.ru_prosience_wget/proscience_index.txt') as ps_index:
    read_index = ps_index.readlines()    

directory_index = '/home/nst/mount/data/linguistics_hse/popular-science-research/polit.ru_prosience_wget'

In [9]:
len(read_index)

1412

In [14]:
def parse_metadata_polit(file_path):
    
        
    with open(file_path) as fl:
        source = fl.read()
        
    re_link = re.compile('.*/polit.ru_prosience_wget\/(.*)index\.html')
    link = re_link.match(file_path)
    link = link.group(1)
    link = ''.join(link)
    
    web_source = 'http://polit.ru/rubric/proscience/'
    
    soup_article = bs.BeautifulSoup(source, 'lxml')
    
    head_title = soup_article.find('h1', class_='title').text
    
    try:    
        author = soup_article.find('div', class_ = 'authors').text.strip()
    except AttributeError:
        author = ''
    
    subtitle = ''

    rubrics = ''
    _rubrics = soup_article.find_all('div', class_= 'tags-subject')
    for rub_href in _rubrics:
        rub_tag = rub_href.select('a[href]')
        for tag in rub_tag:
            rubrics = rubrics + '_' + tag.get_text().lower()  

    broad_rubrics = ''
    

    raw_date = soup_article.find('div', class_='date')
    pre_date = re.search(r'.*([0-9]{2}\s[а-я]+\s[0-9]{4}).*', str(raw_date))
    date = pre_date.group(1)
   

    df = pd.DataFrame([[link, web_source, date, head_title, subtitle, author,
                        broad_rubrics, rubrics]], columns=column_names)
    
    #ready_df = ready_df.append(df)
    return df

In [26]:
data_frame = pd.DataFrame()
df_prosience_articles = pd.DataFrame(data_frame, columns=column_names)
for line in tqdm(read_index):
    line = line[1:]
    directory = directory_index + line.rstrip()
    data_frame = parse_metadata_polit(directory)
    df_prosience_articles = df_prosience_articles.append(data_frame)

A Jupyter Widget

In [59]:
unique_tags = []
tags_list = df_prosience_articles['tags'].values
tags_list = tags_list.tolist()
for line in tags_list:
    split_tags = line.split('_')
    split_tags = split_tags[1:]
    unique_tags.extend(split_tags)
unique_tags = set(unique_tags)
unique_tags = '\n'.join(unique_tags)

In [60]:
with open('proscience_articles_tag_list.txt', 'w') as fl:
    fl.write(unique_tags)

In [257]:
with open('proscience_articles.csv', 'w') as fl:
        fl.write(df_prosience_articles)

## ProScience News ##

In [10]:
with open('/home/nst/mount/data/linguistics_hse/popular-science-research/crawlers/pro_science/for news/proscience_index_news.txt') as ps_index:
    read_index = ps_index.readlines()    

directory_index = '/home/nst/mount/data/linguistics_hse/popular-science-research/polit.ru_prosience_wget/polit.ru/'

In [11]:
len(read_index)

3849

In [15]:
data_frame = pd.DataFrame()
df_prosience_news = pd.DataFrame(data_frame, columns=column_names)
for line in tqdm(read_index):
    line = line[1:]
    directory = directory_index + line.rstrip()
    data_frame = parse_metadata_polit(directory)
    df_prosience_news = df_prosience_news.append(data_frame)

A Jupyter Widget

In [66]:
unique_tags = []
tags_list = df_prosience_news['tags'].values
tags_list = tags_list.tolist()
for line in tags_list:
    split_tags = line.split('_')
    split_tags = split_tags[1:]
    unique_tags.extend(split_tags)
unique_tags = set(unique_tags)
unique_tags = '\n'.join(unique_tags)

In [67]:
with open('proscience_news_tag_list.txt', 'w') as fl:
    fl.write(unique_tags)

In [18]:
df_prosience_news.to_csv('proscience_news.csv')